In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 req

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the  dataset
df = pd.read_csv('data_for_llm.csv')
df=df[0:15000]

# Convert it
dataset = Dataset.from_pandas(df)


In [ ]:
from transformers import T5Tokenizer

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_function(examples):
    inputs = examples['recipe_name']
    targets = examples['formatted_directions']

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the dataset by mapping it
tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Loading the pre-trained .Which is the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Then we want to define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    output_dir='./results',
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Now initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Finally we can train the model
trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Step,Training Loss
500,1.799900
1000,0.985000
1500,0.922500
2000,0.897800
2500,0.861400
3000,0.845700
3500,0.831600
4000,0.826900
4500,0.826300
5000,0.812100


TrainOutput(global_step=11250, training_loss=0.8641309868706597, metrics={'train_runtime': 2717.7747, 'train_samples_per_second': 16.558, 'train_steps_per_second': 4.139, 'total_flos': 1522595266560000.0, 'train_loss': 0.8641309868706597, 'epoch': 3.0})

In [ ]:
# Save the model
model.save_pretrained('t5-recipe-model')
tokenizer.save_pretrained('t5-recipe-model')




('t5-recipe-model/tokenizer_config.json',
 't5-recipe-model/special_tokens_map.json',
 't5-recipe-model/spiece.model',
 't5-recipe-model/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-recipe-model')
tokenizer = T5Tokenizer.from_pretrained('t5-recipe-model')

# Initialize the pipeline
generator = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Generate directions for a given recipe
recipe_name = "how to prepare Chicken Pesto Paninis"
result = generator(recipe_name, max_length=512)
print(result[0]['generated_text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Can be prepare in 15 m , can be cooked in 1 h and will be ready in 1 h 15 m Preheat oven to 350 degrees F (175 degrees C). Place chicken breasts in a large, deep skillet. , can be cooked in and stir chicken in the hot skillet until browned, about 5 minutes. Drain and discard grease. Stir in the chicken, and cook until the chicken is tender, about 5 minutes. Stir in the chicken, and cook until the chicken is tender, about 5 minutes. Stir in the chicken broth, and cook until the sauce is melted and the juices run clear, about 5 minutes. Stir in the chicken broth, and cook until the sauce is tender, about 5 minutes. Stir in the chicken broth, and cook until the sauce is tender, about 5 minutes. Stir in the chicken broth, and cook until the chicken is tender, about 5 minutes. Stir in the chicken broth, and cook until the chicken is tender, about 5 minutes. Stir in the chicken broth, and cook until the chicken is tender, about 5 minutes.


In [ ]:
import pickle
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load your fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-recipe-model')
tokenizer = T5Tokenizer.from_pretrained('t5-recipe-model')

# Save the model and tokenizer to a file
with open('t5_recommendation_model.pkl', 'wb') as f:
    pickle.dump({
        'model_state_dict': model.state_dict(),
        'tokenizer': tokenizer
    }, f)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from google.colab import files

# Download the pickle file
files.download('t5_recommendation_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

:smile